# JSONField

> `JSONField` in Django lets you **store and query JSON data** directly in your database (PostgreSQL natively; other databases via Django's ORM since 3.1+). It’s **schema-less**, **flexible**, and super useful for **dynamic** or **nested** data where rigid schema isn’t ideal.

- skip_showdoc: true
- skip_exec: true

## 🚀 **1. What is `JSONField`?**

| Feature             | Details                                                             |
|---------------------|---------------------------------------------------------------------|
| **Type**            | A Django field that stores JSON-formatted data                     |
| **Native Support**  | PostgreSQL (native JSON/JSONB support); other databases via Django  |
| **Django Version**  | Available in **Django 3.1+** for all supported DBs (previously PostgreSQL only) |
| **Data Type**       | JSON data: dictionaries, lists, strings, numbers, booleans         |

---

## 🔨 **2. How to Use `JSONField` in Django**

### Basic Usage Example
```python
from django.db import models

class Practitioner(models.Model):
    name = models.CharField(max_length=255)
    settings = models.JSONField(default=dict)  # Store flexible settings
```

✅ The field accepts **Python dictionaries**, **lists**, or **JSON serializable objects**.

---

## 📝 **3. Data You Can Store in `JSONField`**

| Python Type       | JSON Equivalent |
|-------------------|-----------------|
| `dict`            | JSON object     |
| `list`/`tuple`    | JSON array      |
| `str`             | JSON string     |
| `int`/`float`     | JSON number     |
| `bool`            | JSON boolean    |
| `None`            | JSON null       |

---

## ⚙️ **4. `JSONField` Field Arguments**

| Argument      | Purpose                                             |
|---------------|-----------------------------------------------------|
| `default`     | Default value (e.g., `dict`, `list`)               |
| `null`        | Allows storing `NULL` in the database if `True`    |
| `blank`       | Allows empty form submissions (use with `null`)    |
| `validators`  | List of validator functions for JSON content       |
| `encoder`     | Custom JSON encoder (usually unnecessary)          |
| `decoder`     | Custom JSON decoder (rare use)                     |

---

## ✅ **5. CRUD Operations on JSONField**

### Creating / Saving JSONField Data
```python
p = Practitioner.objects.create(
    name="Dr. John",
    settings={
        "theme": "dark",
        "notifications": {
            "email": True,
            "sms": False
        }
    }
)
```

### Updating JSONField Data
```python
p = Practitioner.objects.get(id=1)
p.settings['notifications']['email'] = False
p.save()
```

### Be careful: `.save()` is needed!  
You must explicitly `.save()` the model after changing nested JSON data.

---

## 🔎 **6. Querying `JSONField` (PostgreSQL and Django ORM)**

Django allows you to query **keys and values inside the JSON** directly!

### Key Lookup (`__contains`, `__has_key`)
```python
# Get practitioners where settings contain a key 'theme'
Practitioner.objects.filter(settings__has_key='theme')

# Get practitioners where settings contain a specific key-value pair
Practitioner.objects.filter(settings__theme='dark')

# Check if JSON contains multiple keys
Practitioner.objects.filter(settings__has_keys=['theme', 'notifications'])
```

### Value Queries (`__contains`)
```python
# Practitioners with 'sms': False inside notifications
Practitioner.objects.filter(settings__notifications__sms=False)
```

### Exact Match
```python
# Exact match for the entire JSON blob (rarely useful)
Practitioner.objects.filter(settings={
    "theme": "dark",
    "notifications": {"email": True, "sms": False}
})
```

---

## ⚡ **7. Indexing JSONField for Speed**

For **PostgreSQL**, you can create **GIN indexes** to speed up JSON lookups.

### Example Migration for Index
```python
from django.contrib.postgres.indexes import GinIndex
from django.db import models

class Practitioner(models.Model):
    settings = models.JSONField()

    class Meta:
        indexes = [
            GinIndex(fields=['settings']),
        ]
```

✅ This boosts query performance for operations like `has_key`, `contains`.

---

## 🧰 **8. Validating JSONField Data**

### Custom Validators Example
```python
from django.core.exceptions import ValidationError

def validate_practitioner_settings(value):
    if 'theme' not in value:
        raise ValidationError('Theme is required in settings.')

class Practitioner(models.Model):
    settings = models.JSONField(validators=[validate_practitioner_settings])
```

✅ Django runs validators **on save()** or through **forms/DRF serializers**.

---

## 🔌 **9. JSONField with Django REST Framework (DRF)**

DRF has **built-in support** for `JSONField`.

### Example Serializer
```python
from rest_framework import serializers
from .models import Practitioner

class PractitionerSerializer(serializers.ModelSerializer):
    class Meta:
        model = Practitioner
        fields = '__all__'
```

✅ Validates JSON automatically.  
✅ Customize validation in the `validate_settings()` method:
```python
def validate_settings(self, value):
    if 'theme' not in value:
        raise serializers.ValidationError('Theme is required!')
    return value
```

---

## 🏗️ **10. Complex Structures & Nested Access**

- Nest deeply structured settings:  
```json
{
  "notifications": {
    "email": {
      "enabled": true,
      "frequency": "daily"
    },
    "sms": {
      "enabled": false
    }
  },
  "preferences": {
    "theme": "dark",
    "dashboard_layout": "compact"
  }
}
```

- Access deep keys in ORM queries is **limited**; use **Raw SQL or F expressions** for advanced operations.

---

## 🚀 **11. JSONField Use Cases in Django**

| Use Case                            | Why JSONField Works Well                 |
|-------------------------------------|------------------------------------------|
| **User/Practitioner Settings**      | Dynamic, per-user preferences            |
| **Logging Custom Data**             | Flexible schema for event logs           |
| **E-commerce Product Attributes**   | Variable properties per product          |
| **Surveys & Forms**                 | Save answers without rigid schema        |
| **API Responses/Cache**             | Save external API responses as JSON      |

---

## 🔒 **12. Limitations & Gotchas**

| Issue                               | Explanation                              |
|-------------------------------------|------------------------------------------|
| **No enforced schema**              | JSONField doesn't enforce structure      |
| **Querying complex nested data**    | Can get tricky; limited ORM support      |
| **Validation responsibility**       | You own it (no automatic schema)         |
| **Partial updates**                 | Modify the object and `.save()` fully; no partial ORM updates |
| **Indexing not universal**          | GIN indexes are PostgreSQL-specific      |

---

## ✅ **13. Best Practices**

| Practice                          | Why?                                        |
|-----------------------------------|---------------------------------------------|
| **Set a default** (`default=dict`) | Prevents null errors                       |
| **Validate JSON structure**        | Enforce rules for consistent data          |
| **Index fields (Postgres)**        | Boost query performance                   |
| **Use JSONField for dynamic data only** | Prefer normal fields for fixed schema  |
| **Cache frequent queries**         | JSON queries can be slower on large datasets |

---

## 🔮 **14. Forward-Looking Tips**

| Goal                           | How                                              |
|--------------------------------|--------------------------------------------------|
| **Schema enforcement**         | Use Django validators or 3rd party libraries like `jsonschema` |
| **Version your settings**      | Add `version` fields to track JSON format changes |
| **Audit trail**                | Use Django signals or `django-simple-history`    |
| **API-first forms**            | Serve JSON schemas to drive dynamic frontend forms (React, Vue) |

---

## ✅ **Summary Table**

| Feature        | Supported                   |
|----------------|-----------------------------|
| Flexible Data  | ✅ Store any JSON-serializable data |
| DB Support     | ✅ PostgreSQL (native); ✅ SQLite/MySQL (Django 3.1+) |
| Querying       | ✅ Key and value lookups (limited for deep queries) |
| Indexing       | ✅ GIN indexes (PostgreSQL only) |
| Validation     | ✅ Custom Django validators   |

---